In [ ]:
from simcore_sdk.nodeports.nodeports import PORTS

In [ ]:
import plotly
import plotly.offline as offline
import plotly.figure_factory as ff
import plotly.graph_objs as go
import pandas as pd
import os

offline.init_notebook_mode(connected=True)

column_name_map = {0: {"name":"time [s]"}, 
                   1: {"name":"rate"}, 
                   2: {"name":"demand"}, 
                   3: {"name":"move/(max - min) sympathetic efferent", "scale":1/0.0008}, 
                   4: {"name":"threshold indirect parasympathetic efferent", "scale":-1.0},
                   5: {"name":"direct parasympathetic efferent", "scale":-1/0.0008}, 
                   6: {"name":"recruitment = coefficient of variation"}}
data_path = PORTS.inputs[0].get()
#data_frame = pd.read_csv(data_path, sep=',', names=[column_name_map[i]["name"] for i in column_name_map.keys()])
columns_filters = range(7)
outputController = pd.read_csv(data_path, sep=' ', header=None, skipinitialspace=True)
# add the column names to some values
column_rename_dict = {}
for i in column_name_map:
    column_rename_dict[i] = column_name_map[i]["name"]
outputController.rename(inplace=True, columns=column_rename_dict)

trace = go.Table(
    header=dict(
        values=[outputController.columns[i] for i in column_name_map.keys()],
        #values=[column_name_map[i]["name"] for i in column_name_map.keys()],
        fill=dict(color="#C2D4FF"),
        align=["left"]
    ),
    cells=dict(
        values=[outputController[outputController.columns[i]] for i in column_name_map.keys()],
        #values=[data_frame[0:8]],
        fill=dict(color="#F5F8FF"),
        align=["left"]
    )
)
data = [trace]

offline.iplot(data, filename='data-table', config={"displayModeBar": False})

In [ ]:
# define variables
NumberOfLayers = 3
_NumberOfCells = [600, 600, 600] # cells per layer
TotalNumberOfCells = sum(_NumberOfCells)
mrows, ncols = outputController.shape
#x=outputController(:,1); %time
#y1=outputController(:,2); %rate
#y2=outputController(:,3); %demand
#y3=outputController(:,4); %move/(max - min)% sympathetic efferent
#y4=-1*outputController(:,5); %threshold % indirect parasympathetic efferent
#y5=-1*outputController(:,6); %direct parasympathetic efferent
#y6=outputController(:,7); %recruitment = coefficient of variation
#y7=outputController(:,8); %first cell
FirstCellPosition=8-1; #initialize where cell information begins

def NumberOfCells(matlab_index):
    return _NumberOfCells[matlab_index-1]

In [ ]:
def create_graph(data_frame, title, x_axis_title, colors=None, lineWidths=None, show_legend=True, names=None):
    if not names:
        names = data_frame.columns
    if not lineWidths:
        lineWidths = [2] * (data_frame.columns.size-1)
    if colors:
        data = [        
            go.Scatter(
                x=data_frame[data_frame.columns[0]],
                y=data_frame[data_frame.columns[i]], 
                opacity=0.5,                        
                name=names[i-1],
                marker=dict(
                    color=colors[i-1]
                ),
                line = dict(
                    color = colors[i-1],
                    width = lineWidths[i-1],
                )
            ) for i in range(1,data_frame.columns.size)
        ]
    else:
        data = [        
            go.Scatter(
                x=data_frame[data_frame.columns[0]],
                y=data_frame[data_frame.columns[i]], 
                opacity=0.5,                        
                name=names[i-1],
            ) for i in range(1,data_frame.columns.size)
        ]

    layout = go.Layout(
        title=title, 
        showlegend=show_legend,
        xaxis=dict(
            title=x_axis_title
        ),
        yaxis=dict(
            #range=[0,.5]    
        ))
    fig = go.Figure(data=data, layout=layout)
    offline.iplot(fig, config={"displayModeBar": False})

In [ ]:
title="Heart Rate(r)"
heart_axis_colums = [0,1,2,3,5]
heart_rate_data = outputController.filter(items=[outputController.columns[i] for i in heart_axis_colums])
heart_rate_data[heart_rate_data.columns[3]] = heart_rate_data[heart_rate_data.columns[3]].apply(lambda x: x/0.0008)
heart_rate_data[heart_rate_data.columns[4]] = heart_rate_data[heart_rate_data.columns[4]].apply(lambda x: x*-1.0/0.0008)
colors = ["magenta", "black", "red", "blue"]
create_graph(data_frame=heart_rate_data, 
             title=title, 
             colors=colors,
             x_axis_title="\u03C4(sec)")

In [ ]:
title = "Parasympathetic Cell Activity: Cardiac Level"
FirstParasympathetic=500-1
start_index = FirstCellPosition+0*TotalNumberOfCells+FirstParasympathetic
final_index = FirstCellPosition+0*TotalNumberOfCells+NumberOfCells(1)-1
parasympathetic_cell_activity_columns = [0]
parasympathetic_cell_activity_columns.extend(range(start_index, final_index))
parasympathetic_cell_activity_data = outputController.filter(items=[outputController.columns[i] for i in parasympathetic_cell_activity_columns])
create_graph(data_frame=parasympathetic_cell_activity_data, 
             title=title, x_axis_title="time(sec)", show_legend=False,
            colors=["blue"]*(len(parasympathetic_cell_activity_columns)-1))

In [ ]:
title = "Cell Activity: Cardiac(r), Intrathoracic(g), Central(b)"
cell_activity_columns = [0]
# cardiac
start_index = FirstCellPosition+0*TotalNumberOfCells
final_index = FirstCellPosition+0*TotalNumberOfCells+NumberOfCells(1)-1
cell_activity_columns.extend(range(start_index, final_index))
colors = ["red"]*(final_index - start_index)
# intrathoracic
start_index = FirstCellPosition+0*TotalNumberOfCells+NumberOfCells(1)
final_index = FirstCellPosition+0*TotalNumberOfCells+NumberOfCells(1)+NumberOfCells(2)-1
cell_activity_columns.extend(range(start_index, final_index))
colors.extend(["green"]*(final_index - start_index))
# central
start_index = FirstCellPosition+0*TotalNumberOfCells+NumberOfCells(1)+NumberOfCells(2)
final_index = FirstCellPosition+0*TotalNumberOfCells+NumberOfCells(1)+NumberOfCells(2)+NumberOfCells(3)-1
cell_activity_columns.extend(range(start_index, final_index))
colors.extend(["blue"]*(final_index - start_index))

cell_activity_data = outputController.filter(items=[outputController.columns[i] for i in cell_activity_columns])
create_graph(data_frame=cell_activity_data, 
             title=title, x_axis_title="time(sec)", show_legend=False,
            colors=colors)

In [ ]:
title = "Heart Throughput: Cardiac(r), Intrathoracic(g), Central(b)"
heart_throughput_columns = [0]
# cardiac
start_index = FirstCellPosition+1*TotalNumberOfCells
final_index = FirstCellPosition+1*TotalNumberOfCells+NumberOfCells(1)-1
heart_throughput_columns.extend(range(start_index, final_index))
colors = ["red"]*(final_index - start_index)
# intrathoracic
start_index = FirstCellPosition+1*TotalNumberOfCells+NumberOfCells(1)
final_index = FirstCellPosition+1*TotalNumberOfCells+NumberOfCells(1)+NumberOfCells(2)-1
heart_throughput_columns.extend(range(start_index, final_index))
colors.extend(["green"]*(final_index - start_index))
# central
start_index = FirstCellPosition+1*TotalNumberOfCells+NumberOfCells(1)+NumberOfCells(2)
final_index = FirstCellPosition+1*TotalNumberOfCells+NumberOfCells(1)+NumberOfCells(2)+NumberOfCells(3)-1
heart_throughput_columns.extend(range(start_index, final_index))
colors.extend(["blue"]*(final_index - start_index))

data = outputController.filter(items=[outputController.columns[i] for i in heart_throughput_columns])
create_graph(data_frame=data, 
             title=title, x_axis_title="time(sec)", show_legend=False,
            colors=colors)

In [ ]:
title = "Blood Throughput: Cardiac(r), Intrathoracic(g), Central(b)"
blood_throughput_columns = [0]
# cardiac
start_index = FirstCellPosition+2*TotalNumberOfCells
final_index = FirstCellPosition+2*TotalNumberOfCells+NumberOfCells(1)-1
blood_throughput_columns.extend(range(start_index, final_index))
colors = ["red"]*(final_index - start_index)
# intrathoracic
start_index = FirstCellPosition+2*TotalNumberOfCells+NumberOfCells(1)
final_index = FirstCellPosition+2*TotalNumberOfCells+NumberOfCells(1)+NumberOfCells(2)-1
blood_throughput_columns.extend(range(start_index, final_index))
colors.extend(["green"]*(final_index - start_index))
# central
start_index = FirstCellPosition+2*TotalNumberOfCells+NumberOfCells(1)+NumberOfCells(2)
final_index = FirstCellPosition+2*TotalNumberOfCells+NumberOfCells(1)+NumberOfCells(2)+NumberOfCells(3)-1
blood_throughput_columns.extend(range(start_index, final_index))
colors.extend(["blue"]*(final_index - start_index))

data = outputController.filter(items=[outputController.columns[i] for i in blood_throughput_columns])
create_graph(data_frame=data, 
             title=title, x_axis_title="time(sec)", show_legend=False,
            colors=colors)

In [ ]:
#carpet plot full network data set
import math
def fix(value):
    if value > 0:
        return math.floor(value)
    return math.ceil(value)

title = "Cell Activity Level, Heart and Blood Throughput {layersnumber} Layers and {cellsperlayer} Cells per Layer'".format(layersnumber=NumberOfLayers, cellsperlayer=" ".join(map(str,_NumberOfCells)))
carpet_plot_columns = [0]
start_index = FirstCellPosition
final_index = FirstCellPosition+TotalNumberOfCells-1
carpet_plot_columns.extend(range(start_index, final_index))

# this does not work with plotly, too much memory involved here
f = lambda x: 50.0*fix(x)
data_scaled = outputController.filter(items=[outputController.columns[i] for i in range(start_index, final_index)]).applymap(f)
data = [   
    
        go.Scattergl(
            x=outputController[outputController.columns[0]],
            y=[i] * data_scaled.columns.size, 
            opacity=0.5,                        
            name=data_scaled.columns[i],
            mode='markers',
            marker=dict(
                size=1,
                color = data_scaled[data_scaled.columns[i]], #set color equal to a variable
                colorscale='Greys',
                showscale=True
            )
        ) for i in range(0,data_scaled.columns.size)
    ]

layout = go.Layout(
    title=title, 
    showlegend=False,
    xaxis=dict(
        title=None
    ),
    yaxis=dict(
        #range=[0,.5]    
    ))
fig = go.Figure(data=data, layout=layout)
offline.iplot(fig, config={"displayModeBar": False})


In [ ]:
#plot layer average neighbour importance (repeated on next figure)
title = "Relative Weight in Hierarchy"
average_neighbour_importance_columns = [0]
start_index = ncols-1-2*NumberOfLayers-NumberOfLayers-1
final_index = ncols-1-2*NumberOfLayers-1
average_neighbour_importance_columns.extend(range(start_index, final_index))

colors = ["rgb({r}, {g}, {b})".format(r=255.0*(1 - i/NumberOfLayers), g=255.0*abs(.5-i/NumberOfLayers), b=255.0*abs(.25-i/NumberOfLayers)) for i in range(1, final_index-start_index + 1)]
lineWidth = [2 + 2 * i/NumberOfLayers for i in range(1, final_index-start_index + 1)]

names = ['Cardiac','Intrathoracic','Central']
data = outputController.filter(items=[outputController.columns[i] for i in average_neighbour_importance_columns])
data.iloc[:,1:] = data.iloc[:,1:].div(outputController.iloc[:,ncols-1], axis=0)
create_graph(data_frame=data, 
             title=title, x_axis_title="time(sec)", show_legend=True,
            colors=colors,
            lineWidths=lineWidth,
            names=names)

In [ ]:
# plot layer neighbour importance standard deviation (shifted by +1 from
# previous graph which plots the layer neighbour importance average
title = "Layer: Neighbour Importance Standard Deviation"
average_neighbour_importance_std_columns = [0]
start_index = ncols-1-2*NumberOfLayers+1-1
final_index = ncols-1-1+1
average_neighbour_importance_std_columns.extend(range(start_index, final_index, 2))

colors = ["rgb({r}, {g}, {b})".format(r=255.0*(1 - i/2.0/NumberOfLayers), g=255.0*(1 - i/2.0/NumberOfLayers), b=255.0*(1 - i/2.0/NumberOfLayers)) for i in range(1, final_index-start_index, 2)]
lineWidth = [1 + 2 * i/NumberOfLayers for i in range(1, final_index-start_index, 2)]
names = ['Cardiac','Intrathoracic','Central']
data = outputController.filter(items=[outputController.columns[i] for i in average_neighbour_importance_std_columns])
create_graph(data_frame=data, 
             title=title, x_axis_title="time(sec)", show_legend=True,
            colors=colors,
            lineWidths=lineWidth,
            names=names)